# **Time Series Homework 2**

Edward Anderson

**Honor Pledge**: On my honor as a student, I have neither given nor received any unauthorized aid on this assignment. - Edward Anderson

In [ ]:
# Libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

## **QUESTION 1: Show a data description.**
1. Provide a summary of the data set to include descriptive statistics.
2. Plot the time series.
3. (optional) Provide any additional plots that may show important characteristics of your data

In [ ]:
# Load & Preprocess Data
sales = pd.read_csv('data/sales_data.csv')
sales['date'] = pd.to_datetime(sales['date'], errors='coerce')
sales['qty'] = pd.to_numeric(sales['qty'], errors='coerce')
sales['val'] = pd.to_numeric(sales['val'], errors='coerce')
# Subset and Group
sales = sales[~sales['world'].isin(['gr', 'na'])]
sales = sales.groupby('date').sum().reset_index().drop(columns=['world'])
sales.sort_values('date', inplace=True)
# Correct sales anaomaly
sales.rename(columns={'qty' : 'order_quantity', 'val' : 'order_value'}, inplace=True)
sales.loc[sales['order_quantity'].idxmax(), 'order_quantity'] = sales.loc[sales['order_quantity'].idxmax() - 1, 'order_quantity']
sales.loc[sales['order_value'].idxmax(), 'order_value'] = sales.loc[sales['order_value'].idxmax() - 1, 'order_value']
# Group by Week
sales['date'] = sales['date'].dt.to_period('W').apply(lambda r: r.start_time)
sales = sales.groupby('date').sum().reset_index()
print(sales.head())

In [ ]:
# Descriptive Stats and EDA
print("Basic Descriptive Statistics:")
sales.describe().style.format("{:,.0f}", subset=['order_quantity', 'order_value'])

The table above shows the descriptive stats of these data. For this assignment, I will be modeling order quantity. These data are weekly, so the date represents the first day of each week. The mean order quantity is ~2900 units, the max is ~16,000, and the minimum is 474. These data are highly variable with a standard deviation of ~2,000 units. Finally, we have plenty of data points (419 weeks) which is enough to detect patterns, trends, and seasonality in the data. 

In [ ]:
# Plot Sales Qty Over Time
plot = px.line(sales, x='date', y='order_quantity', 
               hover_data=['order_quantity'], 
               labels={'order_quantity':'Sales Quantity', 
                       'date':'Date'})
plot.update_layout(title='Sales Over Time')
plot.show()

# Plot Sales Value
plot = px.line(sales, x='date', y='order_value', 
               hover_data=['order_value'], 
               labels={'order_value':'Sales Value', 
                       'date':'Date'})
plot.update_layout(title='Sales Value Over Time')
plot.show()

Based on these plots, there appears to be weak yearly seasonality, and a weak trend as well. 

In [ ]:
# FUCNTIONS
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
def diagnostic_plot(series, title="Time Series"):
    '''Plot Time Series, ACF, PACF, and Histogram for a given series.'''
    plt.style.use('ggplot')
    fig, ax = plt.subplots(2,2, figsize=(15,10))
    fig.suptitle(f'Diagnostic Plots: {title}', fontsize=16, fontweight='bold')
    # Time series plot
    ax[0,0].plot(series, linewidth=1.5, color='darkblue')
    ax[0,0].set_title('Original Time Series', fontweight='bold')
    ax[0,0].set_xlabel('Time')
    ax[0,0].set_ylabel('Value')
    ax[0,0].grid(True, alpha=0.3)
    # ACF plot
    plot_acf(series, lags=min(60, len(series)//2), ax=ax[0,1], alpha=0.05)
    ax[0,1].set_title('Autocorrelation Function (ACF)', fontweight='bold')
    # PACF plot
    plot_pacf(series, lags=min(60, len(series)//2), ax=ax[1, 0], 
              alpha=0.05, method='ywm')
    ax[1, 0].set_title('Partial Autocorrelation Function (PACF)', fontweight='bold')
    
    # Histogram
    ax[1, 1].hist(series, bins=30, density=True, alpha=0.7, 
                    color='darkblue', edgecolor='black')
    ax[1, 1].set_title('Distribution', fontweight='bold')
    ax[1, 1].set_xlabel('Value')
    ax[1, 1].set_ylabel('Density')
    ax[1, 1].grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()

from statsmodels.tsa.stattools import adfuller, kpss
def adf_test(series):
    '''Function to perform ADF test on series and print results.'''
    result = adfuller(series)
    print("ADF TEST RESULTS:")
    print('   ADF Statistic: {:.4f}'.format(result[0]))
    print('   p-value: {:.4f}'.format(result[1]))
    if result[1] < 0.05:
        print("Conclusion: Reject Null Hypothesis -- Series is Stationary")
    else:
        print("Conclusion: Fail to Reject Null Hypothesis -- Series is Non-Stationary")
def kpss_test(series):
    '''function to perform KPSS test on series and print results...'''
    result = kpss(series)
    print("KPSS TEST RESULTS:")
    print('   KPSS Statistic: {:.4f}'.format(result[0]))
    print('   p-value: {:.4f}'.format(result[1]))
    if result[1] < 0.05:
        print("   Conclusion: Reject Null Hypothesis -- Series is Non-Stationary")
    else:
        print("   Conclusion: Fail to Reject Null Hypothesis -- Series is Stationary")
        
from statsmodels.stats.diagnostic import acorr_ljungbox
def ljung_box_test(series, lags=10):
    '''function to perform LB test on series and print results'''
    result = acorr_ljungbox(series, lags=lags, return_df=True)
    print("Ljung-Box Test Results: PVal < .05 means we reject the null and there IS autocorrelation in the data")
    return result.style.format("{:.5f}", subset=['lb_stat', 'lb_pvalue'])

from statsmodels.tsa.seasonal import STL
def stl_decomposition(series, series_name, period):
    '''function to perform STL decompose and plot results'''
    stl = STL(series, period=period)
    result = stl.fit()
    fig = result.plot()
    fig.suptitle(f'STL Decomposition of {series_name}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot Diagnostic Plots -- SALES QUANTITY
diagnostic_plot(sales['order_quantity'], title='Sales Quantity')

The autocorrelation plot slowly decreases, which is a sign of non stationarity. The PACF shows a significant spike at lag 1 and then quickly drops off, which suggests that AR(1) *could* be appropriate for this data. Before we do any modeling, however, I need to ensure that the data is stationary (I don't believe that it is given the ACF plot...).

---

## **QUESTION 2**: (45 points): Apply Box Jenkins Methodology using ARIMA, SARIMA, or SARIMAX depending on your data to obtain and evaluate forecasts. 
1. *Identification*: Perform all the necessary steps for identification and include automatic and manual identification. Summarize the results from your identification process. 
2. *Estimation*: Perform all necessary steps for identification and summarize the results of your estimation process.
3. *Diagnostic Checking*: Perform all necessary steps for diagnostic checking and summarize the results of your diagnostic checking process.
4. *Forecasting*: Compare the performance of the forecasts of your selected models using the appropriate methods on a test data set. Show graphics of the forecast performance to include confidence intervals. Describe your results and conclusions about the usefulness of the models you evaluated to forecast in the application domain of the data set.

### **Part A: Identification**

To identify the appropriate model(s) (and AR / MA orders), I will use ACF / PACF plots, the ADF test, the KPSS test, and the auto_arima function. 

In [ ]:
# Plot Diagnostics
print("ACF / PACF Plots for Sales Quantity: (SAME AS ABOVE)")
diagnostic_plot(sales['order_quantity'], title='Sales Quantity')

Based on the ACF plot, there is a slow decay of autocorrelations at lags 1-60, which is a strong sign of non-stationarity. The PACF plot shows a significant spike at lag 1 and then quickly drops off; however, some of the lags after lag 1 are also significant. The spike at lag 1 suggests that AR(1) could be appropriate for the data. The data likely needs to be differenced, so I will assess the PACF and ACF plots after differencing to determine the appropriate AR and MA orders.

In [ ]:
adf_test(sales['order_quantity']); print("\n")
kpss_test(sales['order_quantity'])

The ADF test and KPSS test conflict with each other. They both reject the null hypothesis. For ADF, this teams that the data are stationary, but for the KPSS test, this means that the data are nonstationary. The P-Value for the ADF test is close to .05, however, so it barely rejects the null hypothesis. Given the ACF plot, I do not believe these data to be stationary, so I will difference the data, and then reassess these diagnostics.  

In [ ]:
# First Difference + Diagnostics
sales['order_quantity_diff'] = sales['order_quantity'].diff()
diagnostic_plot(sales['order_quantity_diff'].dropna(), title='First Difference of Sales Quantity')
adf_test(sales['order_quantity_diff'].dropna()); print("\n")
kpss_test(sales['order_quantity_diff'].dropna())

After taking the first difference, the ACF shows small negative spikes at lag 1-3 and then quickly drops off. This suggests that an MA(3) *could* be appropriate for this data. The PACF shows significant negative spikes at lags 1-4 and then it quickly drops off. This suggests that an AR(4) model *could* be appropriate for these data. Finally, the ADF and KPSS tests both conclude that the series IS stationary after this first difference, which corroborates what I see in the ACF and PACF plots. Lets see if auto_arima agrees with my conclusions...

In [ ]:
import pmdarima as pm
# First try a nonseasonal model
print("Finding best arima based on AIC:")
auto_arima_model = pm.auto_arima(
    sales['order_quantity'], 
    seasonal=False,
    stepwise=True,
    trace=True,
    suppress_warnings=True,
    error_action='ignore', 
    information_criterion='aic'
)
print(auto_arima_model.summary())
print("\n\nFinding best arima based on BIC:")
auto_arima_model = pm.auto_arima(
    sales['order_quantity'], 
    seasonal=False,
    stepwise=True,
    trace=True,
    suppress_warnings=True,
    error_action='ignore', 
    information_criterion='bic'
)
print(auto_arima_model.summary())

The auto-arima model based on AIC suggests that the best model is ARIMA(4,1,3). This is what I observed in the manual diagnostics above as well! Based on BIC, the best ARIMA model is ARIMA(2,1,2). I will also try a seasonal ARIMA with the same AR and MA orders as the nonseasonal model to try to account for the weak yearly seasonality that I observed in the data. 

### **PART B & C: ESTIMATION + DIAGNOSTIC CHECKING**

Fit the model to the data.

Perform all necessary steps for diagnostic checking and summarize the results of your diagnostic checking process.

In [ ]:
# STL Decomposition to Assess Components of Series...
stl_decomposition(sales['order_quantity'], series_name='Sales Quantity', period=52)

Based on the STL decomposition, there is a weak seasonal component. Based on this, I will try a seasonal ARIMA model as well as the nonseasonal ARIMA model below. Residual variance is also higher from weeks 250-300. I might be able to model this with an exogenous variable, as the heightened demand is a lagged result of the COVID pandemic (lagged because demand responded when inventory arrived). 

In [ ]:
# Define Results DF
model_results = {
    'Model': [], 
    'MSE' : [],
    'RMSE' : [],
    'AIC' : [],
    'BIC' : [], 
    'Heteroskedasticity P-Val': [], 
    'Ljung-Box Residual Autocorrelation P-Val' : []
}

In [ ]:
def model_fit_plot(df, date_col, actual_col, fit_col, residuals, title):
    # Diagnostic Plots... Actual vs Fitted, Residuals, ACF/PACF of Residuals, Histogram of Residuals
    import scipy.stats as stats
    plt.style.use('ggplot')
    fig, ax = plt.subplots(3,2, figsize=(15,10))
    fig.suptitle(f'Diagnostic Plots: {title}', fontsize=16, fontweight='bold')
    # Actual vs Fitted
    ax[0,0].plot(df[date_col], df[actual_col], label='Actual', color='blue')
    ax[0,0].plot(df[date_col], df[fit_col], label='Fitted', color='orange')
    ax[0,0].set_title('Actual vs Fitted Values', fontweight='bold')
    ax[0,0].set_xlabel('Date')
    ax[0,0].set_ylabel('Sales Quantity')
    ax[0,0].legend()
    ax[0,0].grid(True, alpha=0.3)
    # Residuals
    ax[0,1].plot(df[date_col], residuals, label='Residuals', color='red')
    ax[0,1].set_title('Residuals Over Time', fontweight='bold')
    ax[0,1].set_xlabel('Date')
    ax[0,1].set_ylabel('Residual Value')
    ax[0,1].legend()
    ax[0,1].grid(True, alpha=0.3)
    # ACF of Residuals
    plot_acf(residuals, lags=min(60, len(residuals)//2), ax=ax[1, 0], alpha=0.05)
    ax[1, 0].set_title('ACF of Residuals', fontweight='bold')
    # PACF of Residuals
    plot_pacf(residuals, lags=min(60, len(residuals)//2), ax=ax[1, 1], alpha=0.05, method='ywm')
    ax[1, 1].set_title('PACF of Residuals', fontweight='bold')
    # QQ plot of residuals
    sm.qqplot(residuals, line='s', ax=ax[2, 0])
    ax[2, 0].set_title('QQ Plot of Residuals', fontweight='bold')
    # Histogram of Residuals
    ax[2, 1].hist(residuals, bins=30, density=True, alpha=0.7, color='red', edgecolor='black')
    mean = np.mean(residuals)
    sigma = np.std(residuals)
    x = np.linspace(mean - 4*sigma, mean + 4*sigma, 1000)
    normal_pdf = stats.norm.pdf(x, mean, sigma)
    ax[2, 1].plot(x, normal_pdf, color='blue', linestyle='--', label='Normal PDF')
    ax[2, 1].legend()
    ax[2, 1].set_title('Histogram of Residuals', fontweight='bold')
    ax[2, 1].set_xlabel('Residual Value')
    ax[2, 1].set_ylabel('Density')
    ax[2, 1].grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()

def model_eval(fit_model):
    # MSE
    residuals = fit_model.resid
    mse = np.mean(residuals**2)
    # RMSE
    rmse = np.sqrt(mse)
    # AIC and BIC
    aic = fit_model.aic
    bic = fit_model.bic
    # Ljung Box Test on Residuals
    ljung_box_results = acorr_ljungbox(residuals, lags=1, return_df=True)
    ljung_box_pval = ljung_box_results['lb_pvalue'].iloc[0]
    # Heteroskedasticity Test on Residuals
    from statsmodels.stats.diagnostic import het_arch
    arch_test = het_arch(residuals, nlags=12)
    arch_pval = arch_test[1]
    return mse, rmse, aic, bic, ljung_box_pval, arch_pval

In [ ]:
# Fit Model
P = 4
D = 1
Q = 3
Y = sales['order_quantity']

# Fit Model and Output Summary
model = ARIMA(Y, order=(P,D,Q))
model_fit = model.fit()
print(model_fit.summary())

# Add fitted values to sales df for diagnostics
FIT_COL = f'ARIMA_{P,D,Q}'
sales[FIT_COL] = model_fit.fittedvalues
residuals = model_fit.resid

# Model Diagnostic Results
model_fit_plot(sales, date_col='date', actual_col='order_quantity', fit_col=FIT_COL, residuals=residuals, title=f'ARIMA({P},{D},{Q})')
mse, rmse, aic, bic, ljung_box_pval, arch_pval = model_eval(model_fit)
model_results['Model'].append(f'ARIMA({P},{D},{Q})')
model_results['MSE'].append(mse)
model_results['RMSE'].append(rmse)
model_results['AIC'].append(aic)
model_results['BIC'].append(bic)
model_results['Ljung-Box Residual Autocorrelation P-Val'].append(ljung_box_pval)
model_results['Heteroskedasticity P-Val'].append(arch_pval)

Overall, this model fit well aside from the slight heteroskedasticity that I see above in the residuals plot. The ACF and PACF of the residuals show no spikes, however, which is a good sign. The histogram shows that the residuals are approximately normally distributed (slightly tighter than normal), which is also a good sign. Finally, the QQ plot shows that the residuals are approximately normally distributed as well.

Based on the heteroskedasticity, I am going to log transform the data and refit the model to see if that helps... 

In [ ]:
# Log Transform Data and Refit Model
sales['log_order_quantity'] = np.log(sales['order_quantity'])
auto_arima_model_log = pm.auto_arima(
    sales['log_order_quantity'],
    seasonal=False,
    stepwise=True,
    trace=True,
    suppress_warnings=True,
    error_action='ignore',
    information_criterion='aic'
)
print(auto_arima_model_log.summary())
diagnostic_plot(sales['log_order_quantity'].diff().dropna(), title='Log of Sales Quantity (First Difference)')

Based on the autoarima above, the best model to try after a log transform is a ARIMA(3,1,1). I differenced the data once, and this corroborates with what I see in the diagnostic plots as well. There are significant ACF spikes at lag 1, which suggests MA(1) would work well, and significant spikes at PACF lag 1-3 suggest that AR(3) would work well (spikes at lag 4-5 are boarderline...).

In [ ]:
# Fit Model
P = 3
D = 1
Q = 1
Y = sales['log_order_quantity']

# Fit Model and Output Summary
model = ARIMA(Y, order=(P,D,Q))
model_fit = model.fit()
print(model_fit.summary())

# Add fitted values to sales df for diagnostics
FIT_COL = f'LOG_ARIMA_{P,D,Q}'
sales[FIT_COL] = model_fit.fittedvalues
residuals = model_fit.resid

# Model Diagnostic Results
model_fit_plot(sales, date_col='date', actual_col='log_order_quantity', fit_col=FIT_COL, residuals=residuals, title=f'LOG ARIMA({P},{D},{Q})')
mse, rmse, aic, bic, ljung_box_pval, arch_pval = model_eval(model_fit)
model_results['Model'].append(f'LOG ARIMA({P},{D},{Q})')
model_results['MSE'].append(mse)
model_results['RMSE'].append(rmse)
model_results['AIC'].append(aic)
model_results['BIC'].append(bic)
model_results['Ljung-Box Residual Autocorrelation P-Val'].append(ljung_box_pval)
model_results['Heteroskedasticity P-Val'].append(arch_pval)

The log transform helped dramatically to reduce the heteroskedasticity and outliers in the residuals. The ACF and PACF of the residuals show no spikes and appear to be white noise, which is a good sign. The histogram is also normally distributed, and the QQ plot shows that the residuals are approximately normally distributed as well. Overall, this model fit is better than the non-log transformed model, so I will move forward with this version. 

I am unsure why the prediction is 0 at T=0. 

In [ ]:
# SARIMA LOG MODEL
P = 3
D = 1
Q = 1
P_S = 1
D_S = 0
Q_S = 1
SEASON = 52
Y = sales['log_order_quantity']

# Fit Model and Output Summary
model = ARIMA(Y, order=(P,D,Q), seasonal_order=(P_S, D_S, Q_S, SEASON))
model_fit = model.fit()
print(model_fit.summary())

# Add fitted values to sales df for diagnostics
FIT_COL = f'LOG_SARIMA_{P,D,Q}x{P_S,D_S,Q_S,SEASON}'
sales[FIT_COL] = model_fit.fittedvalues
residuals = model_fit.resid

# Model Diagnostic Results
model_fit_plot(sales, date_col='date', actual_col='log_order_quantity', fit_col=FIT_COL, residuals=residuals, title=f'LOG SARIMA({P},{D},{Q}x{P_S},{D_S},{Q_S},{SEASON})')
mse, rmse, aic, bic, ljung_box_pval, arch_pval = model_eval(model_fit)
model_results['Model'].append(f'LOG SARIMA({P},{D},{Q}x{P_S},{D_S},{Q_S},{SEASON})')
model_results['MSE'].append(mse)
model_results['RMSE'].append(rmse)
model_results['AIC'].append(aic)
model_results['BIC'].append(bic)
model_results['Ljung-Box Residual Autocorrelation P-Val'].append(ljung_box_pval)
model_results['Heteroskedasticity P-Val'].append(arch_pval)


In [ ]:
# Model Results
model_results_df = pd.DataFrame(model_results)
model_results_df.style

In all cases, the heteroskedasticity P Value is very low, which suggests that there is still some heteroskedasticity in the residuals. This may be due to the odd spike at T=0, of which I am unsure of the cause...

Otherwise, the SARIMA model appears to have the best fit in terms of AIC, MSE, and RMSE. In all cases, the Ljung Box test fails to reject the null hypothesis, which suggests that there is NO autocorrelation in the residuals (they are just white noise). 

I will transform the predictions back to the orginal scale and plot them against the actuals to see how each tracks the data. 

In [ ]:
# Plotting Predictions vs Actual for Each Model
sales.columns.tolist()

### **PART D: FORECASTING**

Compare the performance of the forecasts of your selected models using the appropriate methods on a test data set. Show graphics of the forecast performance to include confidence intervals. Describe your results and conclusions about the usefulness of the models you evaluated to forecast in the application domain of the data set.